In [1]:
import os
import os.path as osp

import numpy as np
import pandas as pd
import wandb

from mmengine.config import Config
from mmdet.apis import DetInferencer
from pycocotools.coco import COCO

In [2]:
# custom 설정
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass",
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

root = "../../dataset/"
test_ann = "test.json"

saved_epoch = 18
batch_size = 4
score_thr = 0.05
device = "cuda:0"

work_dir = "./work_dirs/cascade_rcnn_swin_large"
checkpoint_path = osp.join(work_dir, f"best_coco_bbox_mAP_epoch_12.pth")

# config file 들고오기
cfg = Config.fromfile("configs/my_model/cascade_rcnn_swin_large.py")
cfg.default_scope = "mmdet"

# dataset config 수정
ds = cfg.test_dataloader["dataset"] if "dataset" in cfg.test_dataloader else cfg.test_dataloader
ds.metainfo = dict(classes=classes)
ds.data_root = root
ds.ann_file = test_ann
ds.data_prefix = dict(img="")
cfg.test_dataloader.dataset.pipeline[1]["scale"] = (512, 512)
cfg.visualizer.vis_backends = [dict(type='LocalVisBackend', save_dir='./temp')]
cfg.randomness = dict(seed=2025, deterministic=False)
for i in range(3):
    cfg.model.roi_head.bbox_head[i].num_classes = len(classes)

In [3]:
# test image info
coco = COCO(osp.join(root, test_ann))
img_ids = coco.getImgIds()
imgs = coco.loadImgs(img_ids)

def _img_path(im):
    return osp.join(root, "", im["file_name"]) if "" \
        else osp.join(root, im["file_name"])

img_paths = [_img_path(im) for im in imgs]
file_names = [im["file_name"] for im in imgs]

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [4]:
# DetInferencer를 이용한 batch inference
inferencer = DetInferencer(
    model=cfg,
    weights=checkpoint_path,
    device=device
)

results = inferencer(
    img_paths,
    batch_size=batch_size,
    return_datasamples=True,
    no_save_vis=True,
)



Loads checkpoint by local backend from path: ./work_dirs/cascade_rcnn_swin_large/best_coco_bbox_mAP_epoch_12.pth
12/10 09:29:48 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


Output()

In [5]:
# submission 양식에 맞게 output 후처리
prediction_strings = []

for data_sample in results["predictions"]:
    inst = data_sample.pred_instances
    bboxes = inst.bboxes.cpu().numpy() if hasattr(inst.bboxes, "cpu") else np.asarray(inst.bboxes)
    scores = inst.scores.cpu().numpy() if hasattr(inst.scores, "cpu") else np.asarray(inst.scores)
    labels = inst.labels.cpu().numpy() if hasattr(inst.labels, "cpu") else np.asarray(inst.labels)

    keep = scores >= score_thr
    bboxes, scores, labels = bboxes[keep], scores[keep], labels[keep]

    ps = []
    for (x1, y1, x2, y2), sc, lb in zip(bboxes, scores, labels):
        ps.extend([str(int(lb)), f"{float(sc):.6f}",
                   f"{float(x1):.1f}", f"{float(y1):.1f}",
                   f"{float(x2):.1f}", f"{float(y2):.1f}"])
    prediction_strings.append(" ".join(ps))

submission = pd.DataFrame({
    "PredictionString": prediction_strings,
    "image_id": file_names,
})
save_path = osp.join(work_dir, f"output.csv")
submission.to_csv(save_path, index=False)
print(f"Saved: {save_path}")
print(submission.head())

Saved: ./work_dirs/cascade_rcnn_swin_large/output.csv
                                    PredictionString       image_id
0  7 0.999412 609.7 523.2 942.1 1024.0 7 0.999371...  test/0000.jpg
1  5 0.950367 127.9 0.0 502.7 303.9 5 0.917582 34...  test/0001.jpg
2  1 0.966402 773.7 435.3 1024.0 1024.0 0 0.96597...  test/0002.jpg
3  9 0.999844 149.3 270.5 914.8 814.4 9 0.347040 ...  test/0003.jpg
4  1 0.996591 208.8 281.8 875.0 771.6 1 0.346045 ...  test/0004.jpg
